# Практическое задание к уроку 4. Сверточные нейронные сети

### Практическое задание

#### Вариант 1. (простой)

- обучить сверточную нейронную сеть в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг (conv-pool)-(conv-pool)-...) на датасете fashion-mnist или cifar10

- оценить рост точности при увеличении ширины сети (больше ядер)

- оценить рост точности при увеличении глубины сети (больше слоев)

- (опциционально)сравнить с точностью полносвязной сети для этой выборки

In [1]:
pip install --upgrade pip

     |████████████████████████████████| 2.1 MB 15.7 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [2]:
!pip install tensorflow -latest


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -l


In [3]:
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 23.3 MB/s eta 0:00:00


In [4]:
!pip install mnist

In [5]:
!pip install tensorflow_datasets

In [6]:
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers
import tensorflow.keras as keras
from keras.layers import Dense
# from keras.utils import to_categorical

from tensorflow.keras.utils import to_categorical
from keras.datasets import fashion_mnist

#### Обучение сверточной нейронной сети в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг (conv-pool)-(conv-pool)-...) на датасете fashion-mnist

In [7]:
(x_train, y_train), (x_test, y_test)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [8]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

y_train = to_categorical(y_train, 10) 
y_test = to_categorical(y_test, 10) 

x_train = x_train.reshape(x_train.shape[0], 28,28,1)
x_test = x_test.reshape(x_test.shape[0], 28,28,1)

#### Оценка роста точности при увеличении ширины сети (больше ядер)

In [9]:
results = []

In [ ]:
#по 10 прогонов с каждым из трёх вариантов размера ядер ( 20 - 10 - 5, 10 - 5 - 3, 5 - 3 - 2) 
# с каждым из двух вариантов кол-ва нейронов для сверточного слоя (6 - 16 - 120, 12-32-120)

for i in range (11):
  result = {}

  # инициализация пустой модели
  model = Sequential()

  # первый сверточный слой
  model.add(layers.Conv2D(12, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(28,28,1), padding="same"))

  # второй пуллинговый слой
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

  # третий сверточный слой
  model.add(layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='valid'))

  # четвертый пуллинговый слой
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  # пятый сверточный слой
  model.add(layers.Conv2D(120, kernel_size=(2, 2), strides=(1, 1), activation='relu', padding='valid'))

  # сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
  model.add(layers.Flatten())

  # шестой полносвязный слой
  model.add(layers.Dense(84, activation='relu'))

  # выходной слой с функцией активации softmax
  model.add(layers.Dense(10, activation='softmax'))


  # компилияция модели
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])


  hh = model.fit(x=x_train,y=y_train, epochs=2, batch_size=128, validation_data=(x_test, y_test), verbose=1)

  result['neurons'] = '12, 32, 120'
  result['kernels'] = '5, 3, 2'
  result['Conv_layer'] = 3
  result['MaxPooling_layer'] = 2
  result['accuracy_tr']= hh.history['accuracy'][-1]
  result['accuracy_val']= hh.history['val_accuracy'][-1]

  results.append(result)

Epoch 1/2
469/469 [==============================] - 106s 225ms/step - loss: 1.0542 - accuracy: 0.6340 - val_loss: 0.6815 - val_accuracy: 0.7236
Epoch 2/2
469/469 [==============================] - 98s 209ms/step - loss: 0.6203 - accuracy: 0.7671 - val_loss: 0.6352 - val_accuracy: 0.7803
Epoch 1/2
469/469 [==============================] - 98s 208ms/step - loss: 1.0607 - accuracy: 0.6282 - val_loss: 0.7425 - val_accuracy: 0.7299
Epoch 2/2
469/469 [==============================] - 98s 209ms/step - loss: 0.6483 - accuracy: 0.7574 - val_loss: 0.6899 - val_accuracy: 0.7611
Epoch 1/2
469/469 [==============================] - 99s 210ms/step - loss: 0.9906 - accuracy: 0.6529 - val_loss: 0.6890 - val_accuracy: 0.7515
Epoch 2/2
469/469 [==============================] - 99s 211ms/step - loss: 0.6373 - accuracy: 0.7642 - val_loss: 0.6046 - val_accuracy: 0.7616
Epoch 1/2
469/469 [==============================] - 99s 210ms/step - loss: 1.0152 - accuracy: 0.6406 - val_loss: 0.7763 - val_accuracy

In [11]:
for i in range (11):
  result = {}

  # инициализация пустой модели
  model = Sequential()

  # первый сверточный слой
  model.add(layers.Conv2D(12, kernel_size=(10, 10), strides=(1, 1), activation='relu', input_shape=(28,28,1), padding="same"))

  # второй пуллинговый слой
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

  # третий сверточный слой
  model.add(layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))

  # четвертый пуллинговый слой
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  # пятый сверточный слой
  model.add(layers.Conv2D(120, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='valid'))

  # сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
  model.add(layers.Flatten())

  # шестой полносвязный слой
  model.add(layers.Dense(84, activation='relu'))

  # выходной слой с функцией активации softmax
  model.add(layers.Dense(10, activation='softmax'))


  # компилияция модели
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])


  hh = model.fit(x=x_train,y=y_train, epochs=2, batch_size=128, validation_data=(x_test, y_test), verbose=1)

  result['neurons'] = '12, 32, 120'
  result['kernels'] = '10, 5, 3'
  result['Conv_layer'] = 3
  result['MaxPooling_layer'] = 2
  result['accuracy_tr']= hh.history['accuracy'][-1]
  result['accuracy_val']= hh.history['val_accuracy'][-1]

  results.append(result)

Epoch 1/2
469/469 [==============================] - 178s 379ms/step - loss: 1.0651 - accuracy: 0.6313 - val_loss: 0.8203 - val_accuracy: 0.6710
Epoch 2/2
469/469 [==============================] - 178s 379ms/step - loss: 0.6040 - accuracy: 0.7793 - val_loss: 0.5754 - val_accuracy: 0.7930
Epoch 1/2
469/469 [==============================] - 176s 374ms/step - loss: 1.0769 - accuracy: 0.6322 - val_loss: 0.7125 - val_accuracy: 0.7126
Epoch 2/2
469/469 [==============================] - 179s 381ms/step - loss: 0.5831 - accuracy: 0.7834 - val_loss: 0.5485 - val_accuracy: 0.7969
Epoch 1/2
469/469 [==============================] - 178s 377ms/step - loss: 1.1050 - accuracy: 0.6027 - val_loss: 0.8918 - val_accuracy: 0.6453
Epoch 2/2
469/469 [==============================] - 178s 379ms/step - loss: 0.6680 - accuracy: 0.7517 - val_loss: 0.6247 - val_accuracy: 0.7653
Epoch 1/2
469/469 [==============================] - 178s 379ms/step - loss: 1.0426 - accuracy: 0.6234 - val_loss: 0.7102 - val_ac

In [12]:
for i in range (11):
  result = {}

  # инициализация пустой модели
  model = Sequential()

  # первый сверточный слой
  model.add(layers.Conv2D(12, kernel_size=(20, 20), strides=(1, 1), activation='relu', input_shape=(28,28,1), padding="same"))

  # второй пуллинговый слой
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

  # третий сверточный слой
  model.add(layers.Conv2D(32, kernel_size=(10, 10), strides=(1, 1), activation='relu', padding='valid'))

  # четвертый пуллинговый слой
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  # пятый сверточный слой
  model.add(layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))

  # сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
  model.add(layers.Flatten())

  # шестой полносвязный слой
  model.add(layers.Dense(84, activation='relu'))

  # выходной слой с функцией активации softmax
  model.add(layers.Dense(10, activation='softmax'))


  # компилияция модели
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])


  hh = model.fit(x=x_train,y=y_train, epochs=2, batch_size=128, validation_data=(x_test, y_test), verbose=1)

  result['neurons'] = '12, 32, 120'
  result['kernels'] = '20, 10, 5'
  result['Conv_layer'] = 3
  result['MaxPooling_layer'] = 2
  result['accuracy_tr']= hh.history['accuracy'][-1]
  result['accuracy_val']= hh.history['val_accuracy'][-1]

  results.append(result)

Epoch 1/2
469/469 [==============================] - 460s 980ms/step - loss: 1.0444 - accuracy: 0.6206 - val_loss: 0.7309 - val_accuracy: 0.7289
Epoch 2/2
469/469 [==============================] - 458s 977ms/step - loss: 0.6170 - accuracy: 0.7687 - val_loss: 0.5933 - val_accuracy: 0.7845
Epoch 1/2
469/469 [==============================] - 460s 980ms/step - loss: 0.9797 - accuracy: 0.6396 - val_loss: 0.7191 - val_accuracy: 0.7200
Epoch 2/2
469/469 [==============================] - 458s 977ms/step - loss: 0.5874 - accuracy: 0.7835 - val_loss: 0.5635 - val_accuracy: 0.7914
Epoch 1/2
469/469 [==============================] - 458s 976ms/step - loss: 1.0338 - accuracy: 0.6209 - val_loss: 0.7138 - val_accuracy: 0.7343
Epoch 2/2
469/469 [==============================] - 459s 980ms/step - loss: 0.6142 - accuracy: 0.7761 - val_loss: 0.5758 - val_accuracy: 0.7937
Epoch 1/2
469/469 [==============================] - 460s 980ms/step - loss: 1.0181 - accuracy: 0.6314 - val_loss: 0.6954 - val_ac

#### Оценка роста точности при увеличении глубины сети (больше слоев)

In [13]:
#2с 1п, 4с 3п, 5с 4п
for i in range (11):
  result = {}

  model = Sequential()

  model.add(layers.Conv2D(6, kernel_size=(6, 6), strides=(1, 1), activation='relu', padding="same"))

  model.add(layers.Conv2D(12, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(28,28,1), padding='valid'))

  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

  model.add(layers.Conv2D(32, kernel_size=(4, 4), strides=(1, 1), activation='relu', padding='valid'))

  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  model.add(layers.Conv2D(80, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding='valid'))

  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  model.add(layers.Conv2D(80, kernel_size=(2, 2), strides=(1, 1), activation='relu', padding='valid'))

  
  # сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
  model.add(layers.Flatten())

  # шестой полносвязный слой
  model.add(layers.Dense(84, activation='relu'))

  # выходной слой с функцией активации softmax
  model.add(layers.Dense(10, activation='softmax'))

  # компилияция модели
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])


  hh = model.fit(x=x_train,y=y_train, epochs=2, batch_size=128, validation_data=(x_test, y_test), verbose=1)

  result['neurons'] = '6, 12, 32, 80, 80'
  result['kernels'] = '6, 5, 4, 3, 2'
  result['Conv_layer'] = 5
  result['MaxPooling_layer'] = 3
  result['accuracy_tr']= hh.history['accuracy'][-1]
  result['accuracy_val']= hh.history['val_accuracy'][-1]

  results.append(result)

Epoch 1/2
469/469 [==============================] - 122s 259ms/step - loss: 1.5174 - accuracy: 0.4619 - val_loss: 0.8833 - val_accuracy: 0.6741
Epoch 2/2
469/469 [==============================] - 121s 258ms/step - loss: 0.7645 - accuracy: 0.7104 - val_loss: 0.8132 - val_accuracy: 0.6776
Epoch 1/2
469/469 [==============================] - 123s 259ms/step - loss: 1.6413 - accuracy: 0.4304 - val_loss: 0.8988 - val_accuracy: 0.6627
Epoch 2/2
469/469 [==============================] - 121s 257ms/step - loss: 0.7559 - accuracy: 0.7194 - val_loss: 0.6974 - val_accuracy: 0.7398
Epoch 1/2
469/469 [==============================] - 122s 259ms/step - loss: 1.4680 - accuracy: 0.4916 - val_loss: 0.8598 - val_accuracy: 0.6624
Epoch 2/2
469/469 [==============================] - 121s 258ms/step - loss: 0.7311 - accuracy: 0.7295 - val_loss: 0.7002 - val_accuracy: 0.7420
Epoch 1/2
469/469 [==============================] - 122s 260ms/step - loss: 1.7268 - accuracy: 0.4484 - val_loss: 0.8741 - val_ac

#### Сравнение точности полносвязной сети для этой выборки

In [ ]:
(train_imagesi,train_labels),(test_imagesi,test_labels)=fashion_mnist.load_data()

In [15]:
# Normalize the images.
train_images = (train_imagesi / 127) - 1
test_images = (test_imagesi / 127) - 1

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

In [16]:
for i in range (11):
  result = {}

  model = Sequential()

  model.add(layers.Dense(90, activation='relu', input_shape=(784,)))

  model.add(layers.Dense(90, activation='relu'))

  # выходной слой с функцией активации softmax
  model.add(layers.Dense(10, activation='softmax'))

  # компилияция модели
  model.compile(loss=keras.losses.categorical_crossentropy, optimizer='Nadam', metrics=["accuracy"])


  hh = model.fit(x=train_images,y=to_categorical(train_labels), epochs=2, batch_size=128, validation_data=(test_images, to_categorical(test_labels)), verbose=1)

  result['neurons'] = ''
  result['kernels'] = ''
  result['Conv_layer'] = 0
  result['MaxPooling_layer'] = 0
  result['accuracy_tr']= hh.history['accuracy'][-1]
  result['accuracy_val']= hh.history['val_accuracy'][-1]

  results.append(result)

Epoch 1/2
469/469 [==============================] - 4s 6ms/step - loss: 0.5191 - accuracy: 0.8133 - val_loss: 0.4289 - val_accuracy: 0.8442
Epoch 2/2
469/469 [==============================] - 3s 6ms/step - loss: 0.3739 - accuracy: 0.8639 - val_loss: 0.3924 - val_accuracy: 0.8603
Epoch 1/2
469/469 [==============================] - 4s 7ms/step - loss: 0.5210 - accuracy: 0.8111 - val_loss: 0.4322 - val_accuracy: 0.8468
Epoch 2/2
469/469 [==============================] - 3s 6ms/step - loss: 0.3759 - accuracy: 0.8636 - val_loss: 0.4089 - val_accuracy: 0.8554
Epoch 1/2
469/469 [==============================] - 4s 7ms/step - loss: 0.5194 - accuracy: 0.8149 - val_loss: 0.4706 - val_accuracy: 0.8271
Epoch 2/2
469/469 [==============================] - 3s 6ms/step - loss: 0.3739 - accuracy: 0.8631 - val_loss: 0.3798 - val_accuracy: 0.8640
Epoch 1/2
469/469 [==============================] - 4s 6ms/step - loss: 0.5228 - accuracy: 0.8106 - val_loss: 0.4470 - val_accuracy: 0.8304
Epoch 2/2
469

In [ ]:
result1 = pd.DataFrame(results)

In [18]:
result1.groupby(['neurons', 'kernels','Conv_layer', 'MaxPooling_layer']).mean(['accuracy_val', 'accuracy_tr']).sort_values(by='accuracy_val', ascending=False)

accuracy_tr  \
neurons           kernels       Conv_layer MaxPooling_layer                
                                0          0                    0.864138   
12, 32, 120       20, 10, 5     3          2                    0.777656   
                  10, 5, 3      3          2                    0.774397   
                  5, 3, 2       3          2                    0.758039   
6, 12, 32, 80, 80 6, 5, 4, 3, 2 5          3                    0.729147   

                                                             accuracy_val  
neurons           kernels       Conv_layer MaxPooling_layer                
                                0          0                     0.859964  
12, 32, 120       20, 10, 5     3          2                     0.786173  
                  10, 5, 3      3          2                     0.779555  
                  5, 3, 2       3          2                     0.770082  
6, 12, 32, 80, 80 6, 5, 4, 3, 2 5          3                     0.741245

**Вывод:**

Результат тем лучше, чем меньше сверточных и пуллинг слоев.

#### Вариант 2. (сложный)

- реализовать нейронную сеть или стек из сверток (Concatenate) на сifar10.

- оценить рост точности при увеличении ширины сети (больше ядер), больше нитей.

- оценить рост точности при увеличении глубины сети (больше слоев)